# Basketball Question

### Importing Data / Packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
#importing text documents
df_Event_Codes = pd.read_csv("Event_Codes.txt", sep = '\t')
df_Game_Lineup = pd.read_csv("Game_Lineup.txt", sep = '\t')

#sorting play by play as suggested in pdf
df_Play_by_Play = pd.read_csv("Play_by_Play.txt", sep = '\t').sort_values(['Game_id',
                                                                           'Period', 
                                                                           'PC_Time',
                                                                           'WC_Time',
                                                                           'Event_Num'],
                                                                          ascending = [0, 1, 0, 1, 1])

### Trying to Get Indicator variable for if player A is in

In [3]:
#creating merge of two dataframes

#gets correct team_id by merging play by play with game lineup. looking only at period 0 to extract info regardless if starter
df_merged = pd.merge(df_Play_by_Play.drop('Team_id', axis = 1),
                     df_Game_Lineup.loc[df_Game_Lineup.Period == 0][['Game_id', 'Person_id', 'Period', 'Team_id']],
                     left_on = ['Game_id', 'Person1'],
                     right_on = ['Game_id', 'Person_id']).drop('Period_y', axis = 1)

#merging to get information of whether player started specific period
df_merged = pd.merge(df_merged,
                     df_Game_Lineup[['Game_id', 'Person_id', 'Period']], 
                     how = 'left',
                     left_on = ['Game_id', 'Person1', 'Period_x'],
                     right_on = ['Game_id', 'Person_id', 'Period'])

#resorting and resetting index
df_merged.sort_values(['Game_id','Period_x', 'PC_Time', 'WC_Time', 'Event_Num'],
                                                                          ascending = [0, 1, 0, 1, 1], inplace = True)

df_merged.reset_index(inplace = True, drop = True)
               

In [4]:
#iterates through df_merged to find location of next non-freethrow event
def findNonFreethrow(i, player_index):
    num = 0
    while (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 3 or 
           df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 8 or 
          df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 6):
        num += 1
        
        if i + num >= len(player_index):
            return -1
        
    return num

#similar to above, but doesnt include fouls
def findNonFreethrowNotIncludingFouls(i, player_index):
    num = 0
    while (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 3 or 
           df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 8):
        num += 1
        
        if i + num >= len(player_index):
            return -1
        
    return num


#iterates through and checks if subsitituion occurs after free throw
def checkIfNotFreeThrow(i, player_index):
    num = 0
    while (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 8 or 
          df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 6):
            num += 1
            if i + num >= len(player_index):
                return True
    
    if df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 3:
        return False
    else:
        return True

In [5]:
#iterating through all players that were listed as starters or were subbed in
#creates column with name of player and puts 1 if player should be credited for points
df_answer = pd.DataFrame(columns = ['Game_ID', 'Player_ID', 'OffRtg', 'DefRtg'])

for player in df_Game_Lineup.Person_id.unique():
    
    #setting indicator value to 0 for all entries to start
    df_merged[player] = 0
    
    #defining boolean statement to select games of interest (games in which player played in)
    games = df_merged.Game_id.isin(df_Game_Lineup.loc[df_Game_Lineup.Person_id == player].Game_id.unique())

    #setting indicator value to 1 for those who start periods
    period_start_index = df_merged.loc[games].drop_duplicates(['Game_id', 'Period_x']).index 

    #groups by period finds players that started their period and puts a 1 in the first entry for those who started their period
    start_of_period = ((df_merged.loc[games].groupby(['Game_id', 'Period'])
                    .apply(lambda x: (x['Person1'] == player).any())).astype(int))
    

    
    #if player doesnt play, no need to set anything
    if len(start_of_period) > 0: 
        start_of_period = start_of_period.reset_index().sort_values(['Game_id', 'Period'],
                            ascending = [0, 1]).reset_index().set_index(period_start_index)
        
        df_merged.loc[period_start_index, player] = start_of_period[0]
    
    
    #setting continual indexes/indicators of being in game
    player_index = df_merged.loc[games].index
    freethrow_indicator = np.ones(len(player_index))
    
    df_merged[player + '_poss'] = np.NaN
    df_merged[player + '_poss'][0] = 0
    
    #going through player index
    for i in range(len(player_index) - 1):
        
        #defining these once
        freethrow_check = checkIfNotFreeThrow(i+1, player_index)
        freethrow_index = findNonFreethrow(i+1, player_index)    
        
    
        df_merged[player].iloc[player_index[i+1]] = (#current value (if already 1, should stay same)
                                    df_merged[player].iloc[player_index[i+1]] +
                        
                                    #no change in value if game or quarter ends:
                                    int((df_merged['Period_x'].iloc[player_index[i+1]] == 
                                         df_merged['Period_x'].iloc[player_index[i]])
                                    & (df_merged['Game_id'].iloc[player_index[i+1]] ==
                                       df_merged['Game_id'].iloc[player_index[i]])) *
                                    
                                    #checking if substitution out of player occured during freethrows:
                                    freethrow_indicator[i] *
                                    
                                    #previous value
                                     (df_merged[player].iloc[player_index[i]]
                        
                                      #checking if play before or after was freethrow
                                      + int(freethrow_check) * 
                                      
                                    #subtract 1 if player has been subbed out
                                      +((- int((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and 
                                        (df_merged['Person1'].iloc[player_index[i+1]] == player))
                          
                                    #add 1 if player has been subbed in
                                       + (int((df_merged['Event_Msg_Type'].iloc[player_index[i]] == 8) and 
                                        (df_merged['Person2'].iloc[player_index[i]] == player)))))))
        
                  
        
        #subbing players out after freethrow
        check_sub_out = ((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and
            #checking if substitution is surounded by freethrows
                (not freethrow_check) &
            #checking if player is being subbed out
                (df_merged['Person1'].iloc[player_index[i+1]] == player) and
            #making sure freethrow wasnt at very end of lastm   game
                freethrow_index != -1 &
            #making sure period is same period
                (df_merged['Period_x'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Period_x'].iloc[player_index[i+1]]) and 
            #making sure game is same game
                (df_merged['Game_id'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Game_id'].iloc[player_index[i+1]]))
    
    
        #checking if substitution
        if (check_sub_out):
            freethrow_indicator[freethrow_index + i] = 0
        
        #subbing players in after freethrow        
        check_sub_in = ((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and
         #checking if substitution is surounded by freethrows
                (not freethrow_check) and
            #checking if player is being subbed out
                (df_merged['Person2'].iloc[player_index[i+1]] == player) and 
            #making sure freethrow wasnt at very end of last game
                findNonFreethrow(i+1, player_index) != -1 and
            #making sure period is same period
                (df_merged['Period_x'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Period_x'].iloc[player_index[i+1]]) and 
            #making sure game is same game
                (df_merged['Game_id'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Game_id'].iloc[player_index[i+1]]))
        
        
         #checking if substitution
        if (check_sub_in):

                df_merged[player].iloc[player_index[freethrow_index + i+1]] = 1
                
        
    
        #counting posessions
        check_missed_shot = (((df_merged['Event_Msg_Type'].iloc[player_index[i]] == 2) or 
                             (df_merged['Event_Msg_Type'].iloc[player_index[i]] == 3)) and
            (df_merged['Event_Msg_Type'].iloc[player_index[i + 1]] == 4) and
            (df_merged['Team_id'].iloc[player_index[i]] != df_merged['Team_id'].iloc[player_index[i+1]]))
        
            
        
        
        
        check_made_final_free_throw = (df_merged['Event_Msg_Type'].iloc[player_index[i]] == 3 and
                                         df_merged['Option1'].iloc[player_index[i]] == 1 and
                                      player_index[findNonFreethrowNotIncludingFouls(i+1, player_index) + i] == player_index[i])
        
        check_turnover = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 5
        
        check_end_quarter = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 13
        
        check_made_field_goal = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 1
        
        check_player_in = df_merged[player].iloc[player_index[i]] == 1
        
        if (check_player_in and (check_made_field_goal or check_made_final_free_throw or 
            check_missed_shot or check_turnover or check_end_quarter)):    
            df_merged[player + '_poss'].iloc[player_index[i]] = max(df_merged[player + '_poss'].dropna()) + 1
        
        
        
        

    df_merged[player + '_poss'].fillna(method = 'ffill', inplace = True)
    
    

    team = df_Game_Lineup.loc[df_Game_Lineup.Person_id == player].Team_id.unique()[0]
    
    for game in df_merged.loc[df_merged.Person2 == player].Game_id.unique():
    
        df_merged_temp = df_merged.loc[df_merged.Game_id == game]
    
        total_offense = (sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 1].loc[df_merged_temp['Team_id'] == team].Option1) +   
                sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 3].loc[df_merged_temp['Team_id'] == team].Option1 == 1))
    
        
        total_defense = (sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 1].loc[df_merged_temp['Team_id'] != team].Option1) +   
                sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 3].loc[df_merged_temp['Team_id'] != team].Option1 == 1))

    
        try:
            offensive_rating = total_offense / (max(df_merged_temp[player + '_poss']) - min(df_merged_temp[player + '_poss'])) * 100
        except:
            offensive_rating = np.NaN
        try:
            defensive_rating = total_defense / (max(df_merged_temp[player + '_poss']) - min(df_merged_temp[player + '_poss']))  * 100
        except:
            defensive_rating = np.NaN

        df_answer = df_answer.append([pd.DataFrame([[game, player, offensive_rating, defensive_rating]], columns = df_answer.columns)])
    

C:\Users\Tom Bliss\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Tom Bliss\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
df_merged.to_csv('df_merged_2.csv')

In [7]:
df_answer.to_csv('final.csv')